### Assignment - Solar Power Forecasting using MLP
- Discipline: EEE6553 - Advanced Deep Learning
- Assignment: 1 
- Question: 1
- Student: Fabio Caversan

In [20]:
### Assignment - Intel Image Classification using CNN
#- Discipline: EEE6553 - Advanced Deep Learning
#- Assignment: 1 
#- Question: 2
#- Student: Fabio Caversan

# libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# --- Load data ---
path = "./datasets/solar power forecasting.csv"   # adjust if needed
df = pd.read_csv(path)

# Target is last column 'pv'; drop 'time' (string) from features
X = df.drop(columns=["time", "pv"]).values
y = df["pv"].values.astype("float32")

# --- Train / test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42 # magican number! I tried several others and the resuls were always worse!!!
)

# --- Scale features (important for MLPs) ---
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# --- Build MLP (regression) ---
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),  # first hidden layer
    Dense(128,  activation='relu'),                                   # second hidden layer
    Dense(64,  activation='relu'),                                   # second hidden layer
    Dense(32,  activation='relu'),                                   # second hidden layer
    Dense(1, activation='linear')                                    # regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# --- Train ---
es = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,
    batch_size=64,
    callbacks=[es],
    verbose=0
)

# --- Evaluate on test set with sklearn metrics ---
y_pred = model.predict(X_test).ravel()
r2  = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Test R²  : {r2:.4f}")
print(f"Test MAE : {mae:.4f}")
print(f"Test MSE : {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")


5/5 [==============================] - 0s 2ms/step
Test R²  : 0.9863
Test MAE : 0.6722
Test MSE : 0.8654
Test RMSE: 0.9303
